In [2]:
%load_ext autoreload
%autoreload 2

### Promblema 
El problema onsiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago de Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo que aterrizó o despegó de SCL.

1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

### acercamiento:
para resolver est punto iniciaremos leyendo los datos, viendo la calidad y el tipo de informacion dentro del dataset. haremos la limpieza necesaria y observaremos que mas podemos hacer con la informacion disponible

In [3]:
# importar librerias
##generacion de path
import latam_airlines.utils.paths as path
## funciones propias
from latam_airlines.utils.latam_utils import check_quality, categorized_days_periods
## manipulacion de datos
import pandas as pd
import numpy as np
## Visualización de los datos
import matplotlib.pyplot as plt
import seaborn as sns

### Lectura y calidad de datos

In [4]:
# Lectura del archivo dataset_SCL.csv
dataset_SCL = pd.read_csv(path.data_raw_dir('dataset_SCL.csv'), low_memory=False)
dataset_SCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Fecha-I    68206 non-null  object 
 1   Vlo-I      68167 non-null  object 
 2   Ori-I      68167 non-null  object 
 3   Des-I      68167 non-null  object 
 4   Emp-I      68167 non-null  object 
 5   Fecha-O    68167 non-null  object 
 6   Vlo-O      68166 non-null  object 
 7   Ori-O      68167 non-null  object 
 8   Des-O      68167 non-null  object 
 9   Emp-O      68167 non-null  object 
 10  DIA        68167 non-null  float64
 11  MES        68167 non-null  float64
 12  AÑO        68167 non-null  float64
 13  DIANOM     68167 non-null  object 
 14  TIPOVUELO  68167 non-null  object 
 15  OPERA      68167 non-null  object 
 16  SIGLAORI   68167 non-null  object 
 17  SIGLADES   68167 non-null  object 
dtypes: float64(3), object(15)
memory usage: 9.4+ MB


observamos que:
- En total hay 68206 datos(ojo la fila 0 cuenta como dato)
- Hay columnas que no tiene todas las filas, indicando la existencia de valores faltantes
- Es necesario transformar las columnas de fecha a formato fecha
- Vemos que no consume mucha uso de memoria, por esta razon crearemos una copia para prevenir futuros problemas al cometer algun error.
- No existe nuestr variable target u objetivo, lo que se traduce en featuring data para crear el target

In [5]:
dataset_SCL_copy = dataset_SCL.copy()
dataset_SCL.sample(10)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
59756,2017-11-26 19:31:00,322,SCEL,SCFA,LAN,2017-11-26 19:43:00,322,SCEL,SCFA,LXP,26.0,11.0,2017.0,Domingo,N,Grupo LATAM,Santiago,Antofagasta
31689,2017-06-28 08:10:00,600,SCEL,SPJC,LAN,2017-06-28 08:09:00,600,SCEL,SPJC,LAN,28.0,6.0,2017.0,Miercoles,I,Grupo LATAM,Santiago,Lima
3030,2017-01-27 18:05:00,152,SCEL,SCSE,LAN,2017-01-27 18:20:00,152,SCEL,SCSE,LAN,27.0,1.0,2017.0,Viernes,N,Grupo LATAM,Santiago,La Serena
46460,2017-09-22 08:55:00,37,SCEL,SCQP,LAN,2017-09-22 08:54:00,37,SCEL,SCQP,LAN,22.0,9.0,2017.0,Viernes,N,Grupo LATAM,Santiago,Temuco
26191,2017-05-11 15:15:00,432,SCEL,SAME,LAN,2017-05-11 15:17:00,432,SCEL,SAME,LAN,11.0,5.0,2017.0,Jueves,I,Grupo LATAM,Santiago,Mendoza
30103,2017-06-07 12:35:00,205,SCEL,SCIE,LAN,2017-06-07 12:36:00,205,SCEL,SCIE,LXP,7.0,6.0,2017.0,Miercoles,N,Grupo LATAM,Santiago,Concepcion
49296,2017-09-17 05:15:00,2370,SCEL,SPJC,LAN,2017-09-17 05:50:00,2370,SCEL,SPJC,LPE,17.0,9.0,2017.0,Domingo,I,Grupo LATAM,Santiago,Lima
36921,2017-07-17 14:55:00,410,SCEL,SUMU,LAN,2017-07-17 15:05:00,410,SCEL,SUMU,LAN,17.0,7.0,2017.0,Lunes,I,Grupo LATAM,Santiago,Montevideo
65122,2017-12-22 21:48:00,188,SCEL,SCAR,LAN,2017-12-22 22:26:00,188.0,SCEL,SCAR,LAN,22.0,12.0,2017.0,Viernes,N,Grupo LATAM,Santiago,Arica
32198,2017-06-20 14:00:00,1103,SCEL,SAEZ,LAN,2017-06-20 14:11:00,1103,SCEL,SAEZ,LAN,20.0,6.0,2017.0,Martes,I,Grupo LATAM,Santiago,Buenos Aires


### Verificacion de calidad de datos

In [6]:
check_quality(dataset_SCL)

Valores NA por columna:
Fecha-I       0
Vlo-I        39
Ori-I        39
Des-I        39
Emp-I        39
Fecha-O      39
Vlo-O        40
Ori-O        39
Des-O        39
Emp-O        39
DIA          39
MES          39
AÑO          39
DIANOM       39
TIPOVUELO    39
OPERA        39
SIGLAORI     39
SIGLADES     39
dtype: int64

Cantidad de datos duplicados:
0

Valores únicos por columna:
Fecha-I      53271
Vlo-I          581
Ori-I            1
Des-I           61
Emp-I           30
Fecha-O      62742
Vlo-O          857
Ori-O            1
Des-O           60
Emp-O           32
DIA             31
MES             12
AÑO              2
DIANOM           7
TIPOVUELO        2
OPERA           23
SIGLAORI         1
SIGLADES        59
dtype: int64


Verificando la calidad de los datos podemos observar:
- existen 39 filas con dainformacion NAN que se pueden iliminar.
- No hay datos duplicados.
- es necesario hacer el cambio de variables string a catgoricas.
- cambiar la columnas fecha a objeto fecha.

In [7]:
# Eliminamos valores NA
dataset_SCL.dropna(inplace=True)

In [8]:
dataset_SCL.sample(10)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
43345,2017-08-10 22:50:00,622,SCEL,MMMX,LAN,2017-08-10 22:50:00,622,SCEL,MMMX,LAN,10.0,8.0,2017.0,Jueves,I,Grupo LATAM,Santiago,Ciudad de Mexico
15961,2017-03-19 08:15:00,449,SCEL,SAEZ,LAN,2017-03-19 08:13:00,449,SCEL,SAEZ,LAN,19.0,3.0,2017.0,Domingo,I,Grupo LATAM,Santiago,Buenos Aires
40874,2017-08-24 02:00:00,97,SCEL,SCCI,LAN,2017-08-24 02:01:00,97,SCEL,SCCI,LAN,24.0,8.0,2017.0,Jueves,N,Grupo LATAM,Santiago,Punta Arenas
67224,2017-12-11 12:30:00,805,SCEL,YMML,LAN,2017-12-11 12:46:00,805.0,SCEL,YMML,LAN,11.0,12.0,2017.0,Lunes,I,Grupo LATAM,Santiago,Melbourne
41698,2017-08-23 09:45:00,233,SCEL,SCQP,LAN,2017-08-23 09:45:00,233,SCEL,SCQP,LXP,23.0,8.0,2017.0,Miercoles,N,Grupo LATAM,Santiago,Temuco
24786,2017-05-18 06:20:00,162,SCEL,SCDA,LAN,2017-05-18 06:23:00,162,SCEL,SCDA,LAN,18.0,5.0,2017.0,Jueves,N,Grupo LATAM,Santiago,Iquique
40202,2017-08-18 17:00:00,134,SCEL,SCCF,SKU,2017-08-18 17:00:00,134,SCEL,SCCF,SKU,18.0,8.0,2017.0,Viernes,N,Sky Airline,Santiago,Calama
1068,2017-01-02 07:50:00,8125,SCEL,SBGL,TAM,2017-01-02 08:07:00,8125,SCEL,SBGL,TAM,2.0,1.0,2017.0,Lunes,I,Grupo LATAM,Santiago,Rio de Janeiro
60074,2017-11-18 16:07:00,386,SCEL,SCAR,LAN,2017-11-18 16:13:00,386,SCEL,SCAR,LXP,18.0,11.0,2017.0,Sabado,N,Grupo LATAM,Santiago,Arica
4781,2017-01-15 15:35:00,445,SCEL,SAEZ,LAN,2017-01-15 15:44:00,445,SCEL,SAEZ,LAN,15.0,1.0,2017.0,Domingo,I,Grupo LATAM,Santiago,Buenos Aires


In [9]:
cols_date = ['Fecha-I','Fecha-O']
for col in cols_date:
    dataset_SCL[col] = pd.to_datetime(dataset_SCL[col], format='%Y-%m-%d %H:%M:%S')

In [10]:
dataset_SCL.describe(datetime_is_numeric=True)

,Fecha-I,Fecha-O,DIA,MES,AÑO
count,68166,68166,68166.000000,68166.000000,68166.000000
mean,2017-07-05 10:37:37.216794368,2017-07-05 10:46:43.755831296,15.714858,6.620441,2017.000029
min,2017-01-01 00:15:00,2017-01-01 00:04:00,1.000000,1.000000,2017.000000
25%,2017-03-31 12:05:00,2017-03-31 12:16:00,8.000000,3.000000,2017.000000
50%,2017-07-10 01:55:00,2017-07-10 01:48:00,16.000000,7.000000,2017.000000
75%,2017-10-08 08:26:00,2017-10-08 08:23:00,23.000000,10.000000,2017.000000
max,2017-12-31 23:55:00,2018-01-01 00:12:00,31.000000,12.000000,2018.000000
std,NaN,NaN,8.782890,3.522743,0.005417


observamos que:
- La resolucion temporal es de un anio de datos, puntualmente el anio 2017.
- No podemos obtener conclusiones o analizar datos con los dias, meses o anios en formato numero, por que son variables categoricas ordinales,esto al poseer un orden natural.
- Debemos crear o reemplazar las columnas de dia, mes y anio(no tan relevante) con sus respectivo datos categoricos.
- Vemos que las demas variables son string y veremos si es necesario cambiarlas a categorias.
- hasta ahora no se eliminaran columns debido que seran utilies mas adelante para hacer procesamientos.

In [25]:
# creamos variables para ayudarnos a hacer el analisis
days = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
months = ['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']
d = dict(zip(np.arange(1, 13),months))
dataset_SCL['MESNOM'] = dataset_SCL['MES'].astype(int).replace(d, regex=True)

#cambiamos las demas variables a formato
cols = ['Vlo-I', 'Ori-I', 'Des-I', 'Emp-I','Vlo-O','Ori-O', 'Des-O', 'Emp-O','DIANOM', 'TIPOVUELO','OPERA', 'SIGLAORI', 'SIGLADES']
for col in cols:
    if col == 'DIANOM':
        dataset_SCL[col] = pd.Categorical(dataset_SCL[col],ordered=True, categories=days)
    else:
        dataset_SCL[col] = dataset_SCL[col].astype('category')

dataset_SCL.sort_values('Fecha-I', inplace=True)

In [26]:
dataset_SCL.sample(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,MESNOM,temporada_alta,dif_min,atraso_15,periodo_dia
42414,2017-08-03 21:13:00,338,SCEL,SCFA,LAN,2017-08-03 21:08:00,338,SCEL,SCFA,LXP,...,Jueves,N,Grupo LATAM,Santiago,Antofagasta,Ago,0,-5.0,0,noche
4307,2017-01-24 18:45:00,320,SCEL,SCAT,LAN,2017-01-24 18:43:00,320,SCEL,SCAT,LXP,...,Martes,N,Grupo LATAM,Santiago,Copiapo,Ene,1,-2.0,0,tarde
12372,2017-03-13 12:45:00,702,SCEL,SAEZ,KLM,2017-03-13 12:40:00,702,SCEL,SAEZ,KLM,...,Lunes,I,K.L.M.,Santiago,Buenos Aires,Mar,0,-5.0,0,tarde
38165,2017-07-11 03:30:00,1165,SCEL,SAEZ,LAN,2017-07-11 03:45:00,1165,SCEL,SAEZ,LAN,...,Martes,I,Grupo LATAM,Santiago,Buenos Aires,Jul,0,15.0,0,noche
49411,2017-09-01 00:30:00,1241,SCEL,SGAS,LAN,2017-09-01 02:20:00,1241,SCEL,SGAS,LAN,...,Viernes,I,Grupo LATAM,Santiago,Asuncion,Sep,0,110.0,1,noche


Con la transformacion de las variables al formato correspondiente iniciamos el analisis de la informacion. Sin embargo, como el objetivo es predecir el retraso en los vuelos de llegada o salida del aeropuerto es fundamenta crear nuestra variable objetivo o target y otras variables auxiliares, ademas, con el fin de no divagar en los datos o obtener metricas que no esten orientadas a responder nuestro objetivo principal(probabilidad de retraso), aqui abordaremos el segundo punto del desafio.

### Problema:
2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :
- temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
- dif_min : diferencia en minutos entre Fecha-O y Fecha-I .
- atraso_15 : 1 si dif_min > 15, 0 si no.
- periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a
Fecha-I .

### Acercamiento:
- Usaremos diversas funciones de pandas y numpy para crear estas nuevas columnas
- Como no hemos hecho un EDA, aprovecharemos la creacion de la variable objetivo para hacer analisis univariados, bivariados y multivariados
- 


In [36]:
# creacion de mascara
temporada_alta = ((dataset_SCL['Fecha-I']>='2017-12-15') | (dataset_SCL['Fecha-I']<='2017-03-04')) | \
    ((dataset_SCL['Fecha-I']>='2017-07-15') & (dataset_SCL['Fecha-I']<='2017-08-01')) | \
        ((dataset_SCL['Fecha-I']>='2017-09-11') & (dataset_SCL['Fecha-I']<='2017-10-01'))

# podemos hacerlo con .loc tambien 
dataset_SCL['temporada_alta'] = np.where(temporada_alta, 1, 0)

# Validación en porcentaje
dataset_SCL['temporada_alta'].value_counts(normalize = True)*100

0    66.618255
1    33.381745
Name: temporada_alta, dtype: float64

In [59]:
# porcentaje de dias en temporada alta
dataset_SCL.loc[dataset_SCL['temporada_alta']==1,'Fecha-I'].dt.date.nunique()/dataset_SCL['Fecha-I'].dt.date.nunique()

0.3178082191780822

### como se hizo
En general, "loc" es más adecuado para seleccionar y manipular filas y columnas específicas en un DataFrame, mientras que "np.where" es más adecuado para aplicar operaciones matemáticas y lógicas complejas a los datos.

En la creacion de la nueva feature 'temporada_alta' usamos una mascara con las condicionales necesarias y nos aprovechamos del conocimiento previo de solo tener un anio(2017).
- Como solo tenemos un anio no es necesario hacer una automatizacion de selecion de fechas o algo por el estilo.
- Como nuestra fecha tiene hasta el formato hora hay que ser precavidos, porque al momento de elegir una fecha menor o igual(<=) el escogera hata las 00 horas de la fecha indicada, asi que debemos agregar un dia mas para seleccione todos los registro de mi fecha limite que va hasta las 23:59 horas, ejemplo: la fecha 3-Mar, debemos colocar en la condicion la fecha 4-Mar, para seleccionar todo el dia 3-mar.
- En la condicion de 1 si Fecha-I está entre 15-Dic y 3-Mar y cambiamos un poco la condicion de 15-Dic o 3-Mar, para que pueda seleccionar el rango de fechas solicitado, esto no afecta el resultado para este caso particular
### que vemos
- Un 33% de los vuelos del anio 2017 fueron hechos en temporada alta
- El 31% o 116 dias del anio estan en temporada alta

In [15]:
# Crear la columna 'dif_min'
dataset_SCL['dif_min'] = (dataset_SCL['Fecha-O'] - dataset_SCL['Fecha-I']).dt.total_seconds() / 60# numero de segundos en un minuto

# Crear la columna 'atraso_15'
dataset_SCL['atraso_15'] =dataset_SCL['dif_min'].map(lambda x: '1' if x>15 else '0')

dataset_SCL['periodo_dia'] = dataset_SCL.apply(categorized_days_periods, axis=1)

In [37]:
dataset_SCL['periodo_dia'].value_counts(normalize = True)*100

tarde     37.414547
mañana    37.137283
noche     25.448171
Name: periodo_dia, dtype: float64

In [16]:
# Validación la correcta creacion de periodo
dataset_SCL['hora']=dataset_SCL['Fecha-I'].dt.hour
dataset_SCL.groupby(by=['periodo_dia','hora']).size()

periodo_dia  hora
mañana       5       2514
             6       4198
             7       4881
             8       4330
             9       3264
             10      2462
             11      3666
noche        0       1082
             1       1573
             2        641
             3        505
             4       1092
             19      3591
             20      2934
             21      2312
             22      2320
             23      1297
tarde        12      3412
             13      3675
             14      3835
             15      3883
             16      3072
             17      3465
             18      4162
dtype: int64

In [17]:
# Eliminar la variable hora
dataset_SCL.drop(columns='hora', inplace=True)

In [18]:
dataset_SCL.sample(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,MESNOM,temporada_alta,dif_min,atraso_15,periodo_dia
10727,2017-02-18 01:40:00,572,SCEL,SKBO,LAN,2017-02-18 01:45:00,572,SCEL,SKBO,LAN,...,Sabado,I,Grupo LATAM,Santiago,Bogota,Feb,1,5.0,0,noche
22073,2017-04-09 18:00:00,43,SCEL,SCIE,LAW,2017-04-09 18:00:00,43,SCEL,SCIE,JMR,...,Domingo,N,Latin American Wings,Santiago,Concepcion,Abr,0,0.0,0,tarde
25515,2017-05-20 05:50:00,281,SCEL,SCCI,LAN,2017-05-20 05:51:00,281,SCEL,SCCI,LXP,...,Sabado,N,Grupo LATAM,Santiago,Punta Arenas,May,0,1.0,0,mañana
33601,2017-07-21 09:05:00,9615,SCEL,SBGL,TAM,2017-07-21 09:23:00,9615,SCEL,SBGL,TAM,...,Viernes,I,Grupo LATAM,Santiago,Rio de Janeiro,Jul,1,18.0,1,mañana
34875,2017-07-09 21:40:00,804,SCEL,SPJC,SKU,2017-07-09 21:38:00,804,SCEL,SPJC,SKU,...,Domingo,I,Sky Airline,Santiago,Lima,Jul,0,-2.0,0,noche


In [ ]:
# Exportar las columnas adicionales a un archivo .csv
dataset_SCL.to_csv('synthetic_features.csv', index=False)